In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import timeit
from datetime import datetime

%load_ext tensorboard

In [2]:
# layers and modules like Keras is built on foundation class tf.Module

class SimpleModule(tf.Module):
    def __init__(self, name=None):
        super(SimpleModule, self).__init__(name=name)
        self.a_variable = tf.Variable(5.0, name='train_me')
        self.non_trainable_variable = tf.Variable(5.0, trainable=False, name='do_not_train')
        self.tensor = tf.constant(2.0)
        
    def __call__(self, x):
        return self.a_variable * x * self.non_trainable_variable
    
simple_module = SimpleModule()
simple_module(tf.constant(5.0))

<tf.Tensor: shape=(), dtype=float32, numpy=125.0>

In [3]:
# All trainable variables
print("trainable variables:", simple_module.trainable_variables)
# Every variable
print("all variables:", simple_module.variables)

trainable variables: (<tf.Variable 'train_me:0' shape=() dtype=float32, numpy=5.0>,)
all variables: (<tf.Variable 'train_me:0' shape=() dtype=float32, numpy=5.0>, <tf.Variable 'do_not_train:0' shape=() dtype=float32, numpy=5.0>)


In [4]:
class Dense(tf.Module):
    def __init__(self, in_features, out_features, name=None):
        super().__init__(name=name)
        self.w = tf.Variable(tf.random.normal([in_features, out_features]), name='w')
        self.b = tf.Variable(tf.zeros([out_features]), name='b')
        
    def __call__(self, x):
        y = tf.matmul(x, self.w) + self.b
        return tf.nn.relu(y)    

class SequentialModule(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)

        self.dense_1 = Dense(in_features=3, out_features=3)
        self.dense_2 = Dense(in_features=3, out_features=2)
        
    def __call__(self, x):
        x = self.dense_1(x)
        return self.dense_2(x)
    
# You have made a model!
my_model = SequentialModule(name="the_model")

# Call it, with random results
print("Model results:", my_model(tf.constant([[2.0, 2.0, 2.0]])))    
    
print("Submodules:", my_model.submodules) 
for var in my_model.variables:
    print(var, "\n")

Model results: tf.Tensor([[0.        0.2809097]], shape=(1, 2), dtype=float32)
Submodules: (<__main__.Dense object at 0x000002265A31EB50>, <__main__.Dense object at 0x000002265A141C10>)
<tf.Variable 'b:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)> 

<tf.Variable 'w:0' shape=(3, 3) dtype=float32, numpy=
array([[ 0.36887783,  0.40947175, -0.40578094],
       [-0.21159618, -0.53556794,  0.43824866],
       [-0.00570034, -0.47715107,  0.20648333]], dtype=float32)> 

<tf.Variable 'b:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)> 

<tf.Variable 'w:0' shape=(3, 2) dtype=float32, numpy=
array([[-0.4960981 ,  0.05518068],
       [ 0.6971016 ,  1.4603413 ],
       [ 0.13769984,  0.552793  ]], dtype=float32)> 



In [5]:
class FlexibleDenseModule(tf.Module):
    # Note: No need for `in+features`
    def __init__(self, out_features, name=None):
        super().__init__(name=name)
        self.is_built = False
        self.out_features = out_features

    def __call__(self, x):
    # Create variables on first call.
        if not self.is_built:
            self.w = tf.Variable(tf.random.normal([x.shape[-1], self.out_features]), name='w')
            self.b = tf.Variable(tf.zeros([self.out_features]), name='b')
            self.is_built = True

        y = tf.matmul(x, self.w) + self.b
        return tf.nn.relu(y)
    
# Used in a module
class MySequentialModule(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)

        self.dense_1 = FlexibleDenseModule(out_features=3)
        self.dense_2 = FlexibleDenseModule(out_features=2)

    def __call__(self, x):
        x = self.dense_1(x)
        return self.dense_2(x)

my_model = MySequentialModule(name="the_model")
print("Model results:", my_model(tf.constant([[2.0, 2.0, 2.0]])))    
    
#This is the reason why tf.keras.layers.Dense also needs only output shape    

Model results: tf.Tensor([[0. 0.]], shape=(1, 2), dtype=float32)


In [6]:
chkp_path = "my_checkpoint"
checkpoint = tf.train.Checkpoint(model=my_model)
checkpoint.write(chkp_path)

'my_checkpoint'

In [7]:
tf.train.list_variables(chkp_path)
new_model = MySequentialModule()
new_checkpoint = tf.train.Checkpoint(model=new_model)
new_checkpoint.restore("my_checkpoint")

# Should be the same result as above
new_model(tf.constant([[2.0, 2.0, 2.0]]))

[('_CHECKPOINTABLE_OBJECT_GRAPH', []),
 ('model/dense_1/b/.ATTRIBUTES/VARIABLE_VALUE', [3]),
 ('model/dense_1/w/.ATTRIBUTES/VARIABLE_VALUE', [3, 3]),
 ('model/dense_2/b/.ATTRIBUTES/VARIABLE_VALUE', [2]),
 ('model/dense_2/w/.ATTRIBUTES/VARIABLE_VALUE', [3, 2])]

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0., 0.]], dtype=float32)>

In [8]:
# Used in a module
class MySequentialModule(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)

        self.dense_1 = FlexibleDenseModule(out_features=3)
        self.dense_2 = FlexibleDenseModule(out_features=2)

    @tf.function    
    def __call__(self, x):
        x = self.dense_1(x)
        return self.dense_2(x)

my_model = MySequentialModule(name="the_model")
print("Model results:", my_model(tf.constant([[2.0, 2.0, 2.0]])))  

Model results: tf.Tensor([[0.        3.8188713]], shape=(1, 2), dtype=float32)


In [9]:
# Set up logging.
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = "logs/func/%s" % stamp
writer = tf.summary.create_file_writer(logdir)

# Create a new model to get a fresh trace
# Otherwise the summary will not see the graph.
new_model = MySequentialModule()

# Bracket the function call with
# tf.summary.trace_on() and tf.summary.trace_export().
tf.summary.trace_on(graph=True)
tf.profiler.experimental.start(logdir)
# Call only one tf.function when tracing.
z = print(new_model(tf.constant([[2.0, 2.0, 2.0]])))
with writer.as_default():
  tf.summary.trace_export(
      name="my_func_trace",
      step=0,
      profiler_outdir=logdir)

tf.Tensor([[0.         0.15735526]], shape=(1, 2), dtype=float32)


In [10]:
%tensorboard --logdir logs/func

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
2021-01-10 01:13:48.623953: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2021-01-10 01:13:48.629499: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Traceback (most recent call last):
  File "C:\Users\kbged\Miniconda3\envs\tf\Scripts\tensorboard-script.py", line 9, in <module>
    sys.exit(run_main())
  File "C:\Users\kbged\AppData\Roaming\Python\Python38\site-packages\tensorboard\main.py", line 71, in run_main
    app.run(tensorboard.main, flags_parser=tensorboard.configure)
  File "C:\Users\kbged\Miniconda3\envs\tf\lib\site-packages\absl\app.py", line 303, in run
    _run_main(main, args)
  File "C:\Users\kbged\Miniconda3\envs\tf\lib\site-packages\absl\app.py", line 251, in _run_main
    sys.exit(main(argv))
  File "C:\Users\kb

In [11]:
tf.saved_model.save(my_model, "the_saved_model")

INFO:tensorflow:Assets written to: the_saved_model\assets


In [12]:
ls -l the_saved_model

 Volume in drive C is Windows
 Volume Serial Number is C41D-C66F

 Directory of C:\Users\kbged\Downloads\tf guide


 Directory of C:\Users\kbged\Downloads\tf guide\the_saved_model

01/10/2021  01:13 AM    <DIR>          .
01/10/2021  01:13 AM    <DIR>          ..
01/02/2021  03:06 PM    <DIR>          assets
01/10/2021  01:13 AM            11,924 saved_model.pb


File Not Found


01/10/2021  01:13 AM    <DIR>          variables
               1 File(s)         11,924 bytes
               4 Dir(s)  13,211,230,208 bytes free


In [13]:
new_model = tf.saved_model.load("the_saved_model")

In [14]:
isinstance(new_model, SequentialModule)

False

In [15]:
class FlexibleDense(tf.keras.layers.Layer):
    # Note the added `**kwargs`, as Keras supports many arguments
    def __init__(self, out_features, **kwargs):
        super().__init__(**kwargs)
        self.out_features = out_features

    def build(self, input_shape):  # Create the state of the layer (weights)
        self.w = tf.Variable(tf.random.normal([input_shape[-1], self.out_features]), name='w')
        self.b = tf.Variable(tf.zeros([self.out_features]), name='b')

    def call(self, inputs):  # Defines the computation from inputs to outputs
        return tf.matmul(inputs, self.w) + self.b

# Create the instance of the layer
flexible_dense = FlexibleDense(out_features=3)

flexible_dense.variables


[]

In [16]:
# Call it, with predictably random results
print("Model results:", flexible_dense(tf.constant([[2.0, 2.0, 2.0], [3.0, 3.0, 3.0]])))

Model results: tf.Tensor(
[[  4.084881    6.4630017  -6.7955956]
 [  6.1273212   9.694503  -10.193394 ]], shape=(2, 3), dtype=float32)


In [17]:
flexible_dense.variables


[<tf.Variable 'flexible_dense/w:0' shape=(3, 3) dtype=float32, numpy=
 array([[ 1.0488793 ,  0.5582854 , -1.3500389 ],
        [-0.30356055,  2.1858964 , -1.8039477 ],
        [ 1.2971218 ,  0.48731896, -0.24381131]], dtype=float32)>,
 <tf.Variable 'flexible_dense/b:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]

In [18]:
try:
  print("Model results:", flexible_dense(tf.constant([[2.0, 2.0, 2.0, 2.0]])))
except tf.errors.InvalidArgumentError as e:
  print("Failed:", e)

Failed: In[0] mismatch In[1] shape: 4 vs. 3: [1,4] [3,3] 0 0 [Op:MatMul]


In [19]:
class MySequentialModel(tf.keras.Model):
  def __init__(self, name=None, **kwargs):
    super().__init__(**kwargs)

    self.dense_1 = FlexibleDense(out_features=3)
    self.dense_2 = FlexibleDense(out_features=2)
  def call(self, x):
    x = self.dense_1(x)
    return self.dense_2(x)

# You have made a Keras model!
my_sequential_model = MySequentialModel(name="the_model")

# Call it on a tensor, with random results
print("Model results:", my_sequential_model(tf.constant([[2.0, 2.0, 2.0]])))
my_sequential_model.variables
my_sequential_model.submodules

Model results: tf.Tensor([[9.255354  1.2252725]], shape=(1, 2), dtype=float32)


[<tf.Variable 'my_sequential_model/flexible_dense_1/w:0' shape=(3, 3) dtype=float32, numpy=
 array([[-1.3889524 ,  0.71009207, -0.27559283],
        [-0.7693984 ,  1.7007347 ,  0.6966048 ],
        [-0.67770296, -1.2339312 , -0.45907006]], dtype=float32)>,
 <tf.Variable 'my_sequential_model/flexible_dense_1/b:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>,
 <tf.Variable 'my_sequential_model/flexible_dense_2/w:0' shape=(3, 2) dtype=float32, numpy=
 array([[-1.2516433 ,  0.16010135],
        [ 0.9333495 ,  0.8826732 ],
        [ 0.5387875 , -0.73251796]], dtype=float32)>,
 <tf.Variable 'my_sequential_model/flexible_dense_2/b:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>]

(<__main__.FlexibleDense at 0x2265a4e3d90>,
 <__main__.FlexibleDense at 0x2265a31bfd0>)

In [20]:
inputs = tf.keras.Input(shape=[3,])

x = FlexibleDense(3)(inputs)
x = FlexibleDense(2)(x)

my_functional_model = tf.keras.Model(inputs=inputs, outputs=x)

my_functional_model.summary()

my_functional_model(tf.constant([[2.0, 2.0, 2.0]]))


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
flexible_dense_3 (FlexibleDe (None, 3)                 12        
_________________________________________________________________
flexible_dense_4 (FlexibleDe (None, 2)                 8         
Total params: 20
Trainable params: 20
Non-trainable params: 0
_________________________________________________________________


<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-4.00458   , -0.87705547]], dtype=float32)>

In [21]:
my_sequential_model.save("exname_of_file")


INFO:tensorflow:Assets written to: exname_of_file\assets


In [22]:
reconstructed_model = tf.keras.models.load_model("exname_of_file")

In [23]:
reconstructed_model(tf.constant([[2.0, 2.0, 2.0]]))

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[9.255354 , 1.2252725]], dtype=float32)>